In [1]:
from typing import Union, List
from pydantic import BaseModel, Field

from langchain_google_vertexai import VertexAI, ChatVertexAI

In [3]:
class Note(BaseModel):
    date: str
    note: str

class Notes(BaseModel):
    # due: List[str]
    # description: List[str]
    notes: List[Note]

In [4]:
def object_to_xml(data: Union[dict, bool], root='object'):
    xml = f'<{root}>'
    if isinstance(data, dict):
        for key, value in data.items():
            xml += object_to_xml(value, key)

    elif isinstance(data, (list, tuple, set)):
        for item in data:
            xml += object_to_xml(item, 'item')

    else:
        xml += str(data)

    xml += f'</{root}>'
    return xml

In [5]:
tasks = [
    {'due':'03/09/2024', 'description': 'submit summer work to physics', 'status': 100},
    {'due':'05/09/2024', 'description': 'request move change to AP US History', 'status':0}
]

In [6]:
xml_tasks = object_to_xml(tasks, 'tasks')

In [8]:
pro = ChatVertexAI(model='gemini-1.5-pro-002').with_structured_output(Notes, method='json_mode')
flash = ChatVertexAI(model='gemini-1.5-pro-002').with_structured_output(Notes, method='json_mode')

In [9]:
USER = 'high school student'

In [18]:
PROMPT = f"""You are generating synthetic data for testing task management application. The application will take informal notes from the user and create list of tasks with due dates.
You need to create list of notes with date on when the note was recorded and actual note text. Create at least 5 notes, some to create task and some to update status of the task. Each note should have at least two tasks
You user is {USER}.
The application should convert those notes into list of tasks:
{xml_tasks}
"""

In [19]:
response = pro.invoke(PROMPT)

In [20]:
response.Notes

[Note(date='02/09/2024', note='Summer work for physics is complete. Need to submit it by tomorrow. Also, request schedule change into AP US History. I need to do it by Tuesday.'),
 Note(date='03/09/2024', note='Physics summer work submitted! AP US History change request in progress. Teacher asked for more information.'),
 Note(date='05/09/2024', note='Received confirmation for schedule change! Officially enrolled in AP US History.'),
 Note(date='08/09/2024', note='First AP US History assignment is due next week. Need to start working on it! Also, need to study for math test by next Friday.'),
 Note(date='12/09/2024', note="Bought new calculator for math. Need to review all formulas before Friday's test."),
 Note(date='15/09/2024', note='Math test done! Prepare physics presentation and practice base guitar for Monday.')]